In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -r drive/MyDrive/Diploma/requirements.txt >& /dev/null

In [3]:
! nvidia-smi

Wed May  5 12:48:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import numpy as np
import torch
from torch import nn

import wandb

from tqdm.notebook import tqdm

from drive.MyDrive.Diploma.environment import KGEnv
from drive.MyDrive.Diploma.utils import create_model, load_config
from drive.MyDrive.Diploma.dataset import KGDataset
from drive.MyDrive.Diploma.train import run_episode, train
from drive.MyDrive.Diploma.beam_search import get_ranks
from drive.MyDrive.Diploma.embed_model import ComplEx
from drive.MyDrive.Diploma.metrics import *

from drive.MyDrive.Diploma.load_config import config

%load_ext autoreload
%autoreload 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
config

{'add_reverse_relations': True,
 'batch_size': 256,
 'beam_size': 128,
 'entities_file': '/content/drive/MyDrive/Diploma/wn18rr/entities.txt',
 'entropy_coef': 0.05,
 'gamma': 0.99,
 'len_penalty': True,
 'lr': 0.0001,
 'max_rollout': 64,
 'normalize_reward': False,
 'num_beam_steps': 10,
 'num_epochs': 2000,
 'num_rollouts': 200,
 'relations_file': '/content/drive/MyDrive/Diploma/wn18rr/relations.txt',
 'rl_method': 'REINFORCE',
 'rs_coef': 1.0,
 'seed': 2441406995705867619}

In [3]:
# torch.manual_seed(config['seed'])
# if torch.cuda.is_available():
#     torch.cuda.manual_seed_all(config['seed'])

In [4]:
train_triplets_path = 'drive/MyDrive/Diploma/wn18rr/train.txt'
test_triplets_path = 'drive/MyDrive/Diploma/wn18rr/test.txt'

In [5]:
env = KGEnv(train_triplets_path)

entities_num = env.entities_num
relations_num = env.relations_num

In [6]:
emb_model = ComplEx(entities_num, relations_num, hid_dim=256).to(device)
emb_model.load_state_dict(torch.load('drive/MyDrive/Diploma/models/emb_model.pt'))

<All keys matched successfully>

In [6]:
batch_size = 256
env = KGEnv(train_triplets_path, batch_size=batch_size, emb_model=None)

agent = create_model(
    entity_input_dim=env.entities_num + 1,  # pad
    relation_input_dim=env.relations_num + 2,  # cls + pad
    output_dim=env.relations_num,
    entity_pad_idx=env.e_pad_idx,
    relation_pad_idx=env.r_pad_idx,
    hid_dim=128,
    enc_pf_dim=256,
    device=device
)

In [12]:
# agent.load_state_dict(torch.load('drive/MyDrive/Diploma/models/pretrained_agent.pt'))

In [7]:
optimizer = torch.optim.Adam(agent.parameters(), lr=1e-4)

In [8]:
wandb.init(project="RL4KGQA", name="REINFORCE")

wandb: Currently logged in as: ashabalin (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.29 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [9]:
torch.initial_seed()

8785330302528065339

In [ ]:
agent.train()

for epoch in tqdm(range(config['num_epochs'])):
    episod_info = run_episode(env, agent)
    loss = train(agent, optimizer, episod_info)

    flat_stops = [prob[0] for probs in episod_info['probs'] for prob in probs]
    # sum_reward = sum([r for reward in episod_info['rewards'] for r in reward])
    wandb.log({'Loss': loss,
               'Accuracy': episod_info['num_correct'] / batch_size,
               'Steps': episod_info['num_steps'],
               'Stop_prob': sum(flat_stops) / len(flat_stops)})

In [ ]:
env = KGEnv(train_triplets_path, batch_size=1)
test_env = KGEnv(test_triplets_path, train=False, batch_size=1)

In [ ]:
train_ranks = get_ranks(agent, env.allowed_triplets, env)
test_ranks = get_ranks(agent, test_env.allowed_triplets, env)

In [ ]:
print('Train | HIT@1: {:.3},\t HIT@10: {:.3},\t MRR: {:.3}'.format(
    hit_k(train_ranks, k=1), hit_k(train_ranks, k=10), mmr(train_ranks)
))
print('Test  | HIT@1: {:.3},\t HIT@10: {:.3},\t MRR: {:.3}'.format(
    hit_k(test_ranks, k=1), hit_k(test_ranks, k=10), mmr(test_ranks)
))

Train | HIT@1: 0.37,	 HIT@10: 0.905,	 MRR: 0.528
Test  | HIT@1: 0.0576,	 HIT@10: 0.503,	 MRR: 0.216


In [ ]:
train_ranks = train_ranks[train_ranks != env.entities_num]
test_ranks = test_ranks[test_ranks != env.entities_num]

In [ ]:
print('Train | HIT@1: {:.3},\t HIT@10: {:.3},\t MRR: {:.3}'.format(
    hit_k(train_ranks, k=1), hit_k(train_ranks, k=10), mmr(train_ranks)
))
print('Test  | HIT@1: {:.3},\t HIT@10: {:.3},\t MRR: {:.3}'.format(
    hit_k(test_ranks, k=1), hit_k(test_ranks, k=10), mmr(test_ranks)
))

Train | HIT@1: 0.378,	 HIT@10: 0.923,	 MRR: 0.539
Test  | HIT@1: 0.0887,	 HIT@10: 0.774,	 MRR: 0.332


In [ ]:
print('Train | HIT@1: {:.3},\t HIT@10: {:.3},\t MRR: {:.3}'.format(
    hit_k(train_ranks, k=1), hit_k(train_ranks, k=10), mmr(train_ranks)
))
print('Test  | HIT@1: {:.3},\t HIT@10: {:.3},\t MRR: {:.3}'.format(
    hit_k(test_ranks, k=1), hit_k(test_ranks, k=10), mmr(test_ranks)
))

Train | HIT@1: 0.205,	 HIT@10: 0.81,	 MMR: 0.381
Test  | HIT@1: 0.0606,	 HIT@10: 0.359,	 MMR: 0.139


In [ ]:
print('Train | HIT@1: {:.3},\t HIT@10: {:.3},\t MRR: {:.3}'.format(
    hit_k(train_ranks, k=1), hit_k(train_ranks, k=10), mmr(train_ranks)
))
print('Test  | HIT@1: {:.3},\t HIT@10: {:.3},\t MRR: {:.3}'.format(
    hit_k(test_ranks, k=1), hit_k(test_ranks, k=10), mmr(test_ranks)
))

Train | HIT@1: 0.625,	 HIT@10: 0.95,	 MMR: 0.737
Test  | HIT@1: 0.184,	 HIT@10: 0.508,	 MMR: 0.294


In [ ]:
np.where(np.array([2, 5, 1, 3]) == 7)[0]

array([], dtype=int64)